# Introduction to dataset

- Size:
- Purpose:
- Properties:
- Created by: 

# Demo in notebook

Just demo with a subset Part1 of the original data consist of 11 parts

## Set up

In [24]:
import os, sys, json
import re
import pandas as pd
import gradio as gr
import numpy as np
import PIL

In [25]:
IMAGE_PATH = "../../NEW_DATASET/SYNTH-PEDES/Part1/"
ANNO_PATH = '../../NEW_DATASET/SYNTH-PEDES/annotation.json'

In [26]:
ids = os.listdir(IMAGE_PATH + 'Part1')
print("Number of IDs: ", len(ids))

Number of IDs:  31231


In [27]:
ANNO = json.load(open(ANNO_PATH))
print(ANNO[0].keys())
ANNO_PART1 = [element for element in ANNO if element['id'] <= 31231]
print(len(ANNO_PART1))

dict_keys(['id', 'file_path', 'attributes', 'captions', 'prompt_caption'])
440200


## View in notebook

In [28]:
df = pd.DataFrame(ANNO_PART1)
df.head(1)

,id,file_path,attributes,captions,prompt_caption
0,1,Part1/1/0.jpg,"[woman,short hair,black jacket,blue denim jean...",[A woman with black hair and she is wearing a ...,[The woman has short hair. She is wearing a bl...


In [66]:
df = pd.DataFrame(ANNO_PART1)
caption_extract_0 = lambda x: x[0]
caption_extract_1 = lambda x: x[1] if len(x) > 1 else None
attribute_convert = lambda x: x[0].replace(',', '<br><br>')
df["caption_0"] = df["captions"].apply(caption_extract_0)
df["caption_1"] = df["captions"].apply(caption_extract_1)
df.drop(columns=["captions"], inplace=True)
df["attributes"] = df["attributes"].apply(attribute_convert)
df["prompt_caption"] = df["prompt_caption"].apply(caption_extract_0)
df.rename(columns={"prompt_caption": "attributes_caption"}, inplace=True)
filepath_convert = lambda x: os.path.join(IMAGE_PATH, x)
df["file_path"] = df["file_path"].apply(filepath_convert)
df.set_index("id", inplace = True)
df.head(2)

,file_path,attributes,attributes_caption,caption_0,caption_1
id,,,,,
1,../../NEW_DATASET/SYNTH-PEDES/Part1/Part1/1/0.jpg,woman<br><br>short hair<br><br>black jacket<br...,The woman has short hair. She is wearing a bla...,A woman with black hair and she is wearing a b...,None
1,../../NEW_DATASET/SYNTH-PEDES/Part1/Part1/1/1.jpg,girl<br><br>dark hair<br><br>black jacket<br><...,A girl with dark hair wearing a black jacket a...,A woman wearing a black jacket over a Gray shi...,None


In [67]:
from IPython.core.display import HTML
def to_img_tag(path):
    return '<img src="' + path + '" width="368" height="768" >'
display(HTML(df.head(50).to_html(escape=False, formatters=dict(file_path=to_img_tag))))

,file_path,attributes,attributes_caption,caption_0,caption_1
id,,,,,
1,,womanshort hairblack jacketblue denim jeansblack sneakersblack backpack,"The woman has short hair. She is wearing a black jacket, blue denim jeans and black sneakers. She is carrying a black backpack.",A woman with black hair and she is wearing a black jacket with blue jeans paired with black shoes.,None
1,,girldark hairblack jacketblue jeansblack shoesunknown,A girl with dark hair wearing a black jacket and blue jeans with black shoes.,"A woman wearing a black jacket over a Gray shirt, a pair of blue jeans and a pair of black shoes.",None
1,,womanbrown hairblack jacketblue jeanswhite shoesblack bag,"The woman is wearing a black jacket, blue jeans and white shoes. She is wearing brown hair and carries a black bag.","A woman wearing a black jacket, a pair of blue jeans and a pair of black and white shoes.","A woman with brown hair is wearing a black jacket, a pair of blue jeans and black and white sneakers."
1,,young womanblack hairblack jacketblue jeansblack shoesblack backpack,A young woman with black hair is wearing a black jacket and she is also carrying a black backpack. She is also wearing blue jeans and black shoes.,A young woman with shoulder-length black hair is wearing a black jacket over a beige shirt. She is also wearing blue fitted jeans and black sneakers.,A young woman with straight shoulder-length black hair is wearing a black jacket over a beige shirt. She is also wearing skinny blue denim pants and black shoes with white soles.
1,,womanbrown hairblack jacketblue jeansblack shoesblack backpack,"The woman is wearing a black jacket, blue jeans and black shoes. She is wearing brown hair and carries a black backpack.",A woman in her thirties with straight long brown hair is wearing a black jacket over a white shirt. She is also wearing skinny blue pants and black shoes with white soles.,None
1,,womanstraight blonde hairblack jacketblue jeansblack shoesunknown,A woman with straight blonde hair is wearing a black jacket paired with blue jeans and black shoes.,A woman in her twenties with straight shoulder length brown hair is wearing a black jacket with blue denim jeans and black sneakers. She is also wearing a black shoulder bag on her shoulder.,None
1,,young womanshoulder-length blonde hairblack jacketblue denim jeansblack shoesblack backpack,A young woman with shoulder-length blonde hair is wearing a black jacket and blue denim jeans. She is also wearing black shoes. She is carrying a black backpack.,A young woman with shoulder-length brown hair is wearing a black jacket and blue denim jeans. She is also wearing black sneakers.,None
1,,young womanbrown hairblack leather jacketblue denim jeansblack shoeswhite paper,A young woman had brown hair and she is wearing a black leather jacket with blue denim jeans She is wearing black shoes.,A young aged woman with shoulder-length brown hair and is wearing a black jacket with blue jeans paired with shoes.,None
1,,young womanshoulder-length blonde hairblack jacketblue jeansblack shoesblack backpack,A young woman with shoulder-length blonde hair is wearing a black jacket paired with blue jeans and black shoes. She is carrying a black backpack.,A young woman with shoulder-length brown hair is wearing a black jacket and dark blue denim jeans. She is also wearing black sneakers with white soles.,None


In [68]:
from sklearn.utils import shuffle
sh_df = shuffle(df)
display(HTML(sh_df.head(300).to_html(escape=False, formatters=dict(file_path=to_img_tag))))

,file_path,attributes,attributes_caption,caption_0,caption_1
id,,,,,
8388,,womanbrown hairbrown overcoatblue jeanswhite shoesblack handbag,A woman with brown hair is wearing a brown overcoat and blue jeans. She is also wearing a pair of white shoes. She is carrying a black handbag.,A woman with a brown cap and is wearing a brown long jacket with blue jeans paired with white shoes.,A woman with a brown hat and is wearing a brown long jacket with blue jeans paired with white shoes.
24711,,middle-aged manblack hairdark blue jacketblack pantsblack shoesunknown,A middle-aged man is wearing a black hair with a matching dark blue jacket. He is also wearing black pants and a pair of black shoes.,A middle-aged man with short black hair is wearing a blue and black zip-up jacket with loose-fitting black jeans. He is pairing it with black shoes.,None
21500,,mancurly grey hairgrey sweatergreen pantsblack shoeswrist watch,A man with curly grey hair is wearing a grey sweater and green pants. He is also wearing black shoes.,"A man wearing a Gray sweater over a white shirt, a pair of green pants and a wrist watch.","A man wearing a Gray sweater over a Gray shirt, a pair of blue pants and a wrist watch."
2101,,womanbrown hairblack shirtblack pantsblack sneakersshoulder bag,"The woman is wearing a black shirt, black pants and black sneakers. She is wearing brown hair and carries a shoulder bag.","A woman wearing a black shirt, a pair of black pants and a pair of black shoes.",None
8957,,womanbrown hairblack coatblack leggingsblack shoesgreen tote bag,A woman with brown hair is wearing a black coat paired with black leggings and black shoes. She is carrying a green tote bag.,"A woman wearing a black jacket, a pair of black shorts and a pair of black and white shoes.","A woman wearing a dark black jacket, a pair of black shorts and a pair of black and white shoes."
16481,,manshort dark hairlong sleeve shirtblack pantsblack shoesunknown,A man having short dark hair is wearing a long sleeve shirt and black pants with black shoes.,"A man wearing a white long sleeve shirt, a pair of black pants and a pair of black shoes.",None
12967,,manshort grey hairblack coatblack pantsblack oxford shoesblack bag,"A man with a short grey hair is wearing a black coat, black pants and black oxford shoes. He is carrying a black bag.",A man in his thirties with short grey hair is wearing a black fleece coat. He is also wearing a pair of black pants.,None
30282,,manblack hairblack jacketblack pantsblack shoeswhite paper,A man with black hair is wearing a black jacket and black pants. He is also wearing black shoes.,A man with black hair and is wearing a black jacket with black pants paired with black shoes carrying a white paper in hand.,None
28134,,middle-aged manblack hairblack jacketblue pantsblack shoeswater bottle,A middle-aged man with black hair is wearing a black jacket. He is also wearing a pair of blue pants and black shoes.,A man with black short hair and is wearing a black jacket with black pants paired with black shoes.,A middle-aged man with short black hair is wearing a black jacket. He is also wearing dark blue denim pants and black shoes with grey soles.


# Interactive GUI

Need to run notebook again!

In [13]:
import random, time
import functools 
import pandas as pd
def generate_random():
    k = random.randint(0, len(ANNO_PART1))
    demo_text = ANNO_PART1[k]['captions']
    demo_text = functools.reduce(lambda acc, x: acc + '\n\n' + x, demo_text) 
    demo_img = PIL.Image.open(IMAGE_PATH+ANNO_PART1[k]['file_path']).convert('RGB').resize((128,384))
    demo_attr = pd.DataFrame(ANNO_PART1[k]['attributes'][0].split(','))
    demo_prompt = ANNO_PART1[k]['prompt_caption']
    return demo_text, demo_img, demo_attr, demo_prompt

In [ ]:
with gr.Blocks() as demo:
    with gr.Row():
        summit_button_0 = gr.Button(value="Click to show image")
    with gr.Row():
        with gr.Column():
            random_text = gr.Textbox(label="Text caption of the image")
            random_prompt = gr.Textbox(label="Attribute-Prompt caption of the image")
        random_image = gr.Image(type="pil", height="384", width="128", label="Image")
    # with gr.Row():
        random_attr = gr.Dataframe(label="Attribute phrases")
    summit_button_0.click(generate_random, inputs=None, outputs=[random_text, random_image, random_attr, random_prompt])
demo.launch(inline=True, share=True)